In [1]:
import numpy as np 
import pandas as pd 
import os

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
!pip install protobuf>=4.21.0

In [9]:
# Check current protobuf version and force uninstall/reinstall
!pip uninstall protobuf -y
!pip install protobuf>=4.25.0

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3


You can safely remove it manually.


In [ ]:
# Force reinstall TensorFlow with compatible protobuf
!pip uninstall tensorflow -y
!pip install tensorflow==2.18.0 protobuf==4.25.3

Found existing installation: tensorflow 2.20.0
Uninstalling tensorflow-2.20.0:
  Successfully uninstalled tensorflow-2.20.0


You can safely remove it manually.


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

AttributeError: module 'google.protobuf.internal.api_implementation' has no attribute '_c_module'